<a href="https://colab.research.google.com/github/SergioAaron/Tiendas/blob/main/Tiendas_2018_2019_en_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S R
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

DATA_SOURCE_MAPPING = 'tiendas-departamentales-2018-2019:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4566990%2F7799904%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240313%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240313T130155Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D49e2edd8ad1d73141f17a1f2dbb8c77f5d288c35dbe9ec20627fd456f1496520b70c0e1ba81465c08e1ebc32ae6483fb4f10b7030218011bf9a83750cfeb6ead75e6fd91f3dbe1033362c8674a03cf31efd700be35de21561efd3544754b06f718248319cc0428c753e0e2462f42b3bd898e65548b6b1b75f0498e6479cfeed804602e11644866486d4ea601fdcf44abecbf7da4d4a525a62342cd8e7b58bda8df4b2251118cef426ce83eb5dde655b3dfbcb7c5dbe22c287c6e70f129f2ca4e3c7a65ad9c9c23f056c56a1dc9b5969674089f070182753a957c4553c83ee865a2dc6fdf23d448970248f9ecd494271d77484e9d836d20b7f6db9f800f4e025d'

KAGGLE_INPUT_PATH = '/kaggle/input'
KAGGLE_WORKING_PATH = '/kaggle/working'

system(paste0('sudo umount ', '/kaggle/input'))
system(paste0('sudo rmdir ', '/kaggle/input'))
system(paste0('sudo mkdir -p -- ', KAGGLE_INPUT_PATH), intern=TRUE)
system(paste0('sudo chmod 777 ', KAGGLE_INPUT_PATH), intern=TRUE)
system(
  paste0('sudo ln -sfn ', KAGGLE_INPUT_PATH,' ',file.path('..', 'input')),
  intern=TRUE)

system(paste0('sudo mkdir -p -- ', KAGGLE_WORKING_PATH), intern=TRUE)
system(paste0('sudo chmod 777 ', KAGGLE_WORKING_PATH), intern=TRUE)
system(
  paste0('sudo ln -sfn ', KAGGLE_WORKING_PATH, ' ', file.path('..', 'working')),
  intern=TRUE)

data_source_mappings = strsplit(DATA_SOURCE_MAPPING, ',')[[1]]
for (data_source_mapping in data_source_mappings) {
    path_and_url = strsplit(data_source_mapping, ':')
    directory = path_and_url[[1]][1]
    download_url = URLdecode(path_and_url[[1]][2])
    filename = sub("\\?.+", "", download_url)
    destination_path = file.path(KAGGLE_INPUT_PATH, directory)
    print(paste0('Downloading and uncompressing: ', directory))
    if (endsWith(filename, '.zip')){
      temp = tempfile(fileext = '.zip')
      download.file(download_url, temp)
      unzip(temp, overwrite = TRUE, exdir = destination_path)
      unlink(temp)
    }
    else{
      temp = tempfile(fileext = '.tar')
      download.file(download_url, temp)
      untar(temp, exdir = destination_path)
      unlink(temp)
    }
    print(paste0('Downloaded and uncompressed: ', directory))
}

print(paste0('Data source import complete'))


In [ ]:
# Importar la librería
library(readxl)
library(dplyr)
library(ggplot2)

# Cargar el archivo Excel
tiendas <- read_excel("/kaggle/input/tiendas-departamentales-2018-2019/MKT Science TR JR - Tiendas Departamentales 2018-2019 (1).xls")

# Explorar las primeras filas del DataFrame
head(tiendas)

# Ver las dimensiones del DataFrame original
dim(tiendas)

# Eliminar filas duplicadas
tiendas_sin_duplicados <- unique(tiendas)

# Ver las dimensiones del DataFrame sin duplicados
dim(tiendas_sin_duplicados)

# Realizar análisis exploratorio de datos (EDA)
summary(tiendas_sin_duplicados) # Resumen estadístico
str(tiendas_sin_duplicados)     # Estructura del DataFrame


#  Inversión Total y Crecimiento Porcentual (2018-2019)

In [ ]:
# Filtrar los datos para incluir solo los años 2018 y 2019
tiendas_2018 <- tiendas_sin_duplicados[tiendas_sin_duplicados$Año == 2018, ]
tiendas_2019 <- tiendas_sin_duplicados[tiendas_sin_duplicados$Año == 2019, ]

# Calcular la inversión total de la categoría para cada año
inversion_total_2018 <- sum(tiendas_2018$Inversión_Bruta)
inversion_total_2019 <- sum(tiendas_2019$Inversión_Bruta)

# Calcular la diferencia entre la inversión total de 2019 y 2018
diferencia_inversion <- inversion_total_2019 - inversion_total_2018

# Calcular la diferencia porcentual entre 2019 y 2018
diferencia_porcentual <- ((inversion_total_2019 - inversion_total_2018) / inversion_total_2018) * 100

# Mostrar los resultados
print(paste("La inversión total en la categoría en 2018 fue: $", inversion_total_2018))
print(paste("La inversión total en la categoría en 2019 fue: $", inversion_total_2019))
print(paste("La diferencia de inversión entre 2019 y 2018 fue: $", diferencia_inversion))
print(paste("La diferencia porcentual entre 2019 y 2018 fue:", round(diferencia_porcentual, 2), "%"))


# Temporalidades Clave de la categoria

## Trimestre

In [ ]:
# Calcular la inversión total trimestral para 2018
inversion_total_trimestral_2018 <- tiendas_sin_duplicados %>%
  filter(format(as.Date(Fecha), "%Y") == "2018") %>%
  group_by(quarters(as.Date(Fecha))) %>%
  summarise(inversion_total = sum(Inversión_Bruta))

print("Inversión Total Trimestral para 2018:")
print(inversion_total_trimestral_2018)

# Calcular la inversión total trimestral para 2019
inversion_total_trimestral_2019 <- tiendas_sin_duplicados %>%
  filter(format(as.Date(Fecha), "%Y") == "2019") %>%
  group_by(quarters(as.Date(Fecha))) %>%
  summarise(inversion_total = sum(Inversión_Bruta))

print("Inversión Total Trimestral para 2019:")
print(inversion_total_trimestral_2019)


## Mensual

In [ ]:
# Calcular la inversión total mensual para 2018
inversion_total_mensual_2018 <- tiendas_sin_duplicados %>%
  filter(format(as.Date(Fecha), "%Y") == "2018") %>%
  group_by(month(as.Date(Fecha))) %>%
  summarise(inversion_total = sum(Inversión_Bruta))

print("Inversión Total Mensual para 2018:")
print(inversion_total_mensual_2018)

# Calcular la inversión total mensual para 2019
inversion_total_mensual_2019 <- tiendas_sin_duplicados %>%
  filter(format(as.Date(Fecha), "%Y") == "2019") %>%
  group_by(month(as.Date(Fecha))) %>%
  summarise(inversion_total = sum(Inversión_Bruta))

print("Inversión Total Mensual para 2019:")
print(inversion_total_mensual_2019)


## Semanal

In [ ]:
# Filtrar datos por año 2018 y calcular la inversión semanal
inversion_semanal_2018 <- aggregate(tiendas_sin_duplicados$Inversión_Bruta[year(tiendas_sin_duplicados$Fecha) == 2018],
                                    list(weekdays(tiendas_sin_duplicados$Fecha[year(tiendas_sin_duplicados$Fecha) == 2018])), sum)

# Filtrar datos por año 2019 y calcular la inversión semanal
inversion_semanal_2019 <- aggregate(tiendas_sin_duplicados$Inversión_Bruta[year(tiendas_sin_duplicados$Fecha) == 2019],
                                    list(weekdays(tiendas_sin_duplicados$Fecha[year(tiendas_sin_duplicados$Fecha) == 2019])), sum)

# Mapear números de día de la semana a nombres
nombres_dias_semana <- c('Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo')

# Mostrar la inversión semanal para el año 2018
cat("Inversión Publicitaria Semanal - 2018:\n")
for (i in 1:nrow(inversion_semanal_2018)) {
  cat(nombres_dias_semana[i], ':', inversion_semanal_2018[i, 2], '\n')
}

# Mostrar la inversión semanal para el año 2019
cat("\nInversión Publicitaria Semanal - 2019:\n")
for (i in 1:nrow(inversion_semanal_2019)) {
  cat(nombres_dias_semana[i], ':', inversion_semanal_2019[i, 2], '\n')
}


# Ranking de Marcas en 2019 y Estrategia de Medios

In [ ]:
# Filtrar los datos para incluir solo el año 2019
tiendas_2019 <- tiendas_sin_duplicados[tiendas_sin_duplicados$Año == 2019, ]

# Agrupar por marca y sumar la inversión bruta para cada marca en 2019
ranking_marcas_2019 <- tiendas_2019 %>%
  group_by(Marca) %>%
  summarise(inversion_total = sum(Inversión_Bruta)) %>%
  arrange(desc(inversion_total))  # Ordenar por inversión total en orden descendente

# Mostrar el ranking de marcas en 2019
print("Ranking de Marcas en 2019:")
print(ranking_marcas_2019)


## Los medios que usaron las tres marcas más importantes durante el año 2019

In [ ]:
# Obtener las tres marcas más importantes del ranking de 2019
top_marcas_2019 <- head(ranking_marcas_2019, 3)

# Filtrar los datos para incluir solo las inversiones realizadas por las tres marcas más importantes
datos_top_marcas_2019 <- tiendas_2019 %>%
  filter(Marca %in% top_marcas_2019$Marca)

# Agrupar por medio y sumar la inversión bruta para cada uno de ellos
inversion_por_medio <- datos_top_marcas_2019 %>%
  group_by(Medio) %>%
  summarise(inversion_total = sum(Inversión_Bruta))

# Mostrar la inversión por medio para las tres marcas más importantes en 2019
print("Inversión por Medio para las Tres Marcas Más Importantes en 2019:")
print(inversion_por_medio)


In [ ]:
# Obtener las tres marcas más importantes
tres_marcas_mas_importantes <- head(ranking_marcas_2019, 3)

# Mostrar las tres marcas más importantes
cat("\nTres Marcas Más Importantes en 2019:\n")
print(tres_marcas_mas_importantes)

# Filtrar los datos para incluir solo las tres marcas más importantes
datos_tres_marcas <- tiendas_2019 %>%
  filter(Marca %in% tres_marcas_mas_importantes$Marca)

# Calcular la inversión por medio para las tres marcas más importantes en 2019
inversion_por_medio <- datos_tres_marcas %>%
  group_by(Marca, Medio) %>%
  summarise(inversion_total = sum(Inversión_Bruta), .groups = "drop")

# Mostrar la inversión por medio para las tres marcas más importantes en 2019
cat("\nInversión por Medio para las Tres Marcas Más Importantes en 2019:\n")
print(inversion_por_medio)


#  Campañas más importantes del 2018

In [ ]:
# Filtrar los datos para incluir solo el año 2018
tiendas_2018 <- tiendas_sin_duplicados[tiendas_sin_duplicados$Año == 2018, ]

# Agrupar los datos por el campo 'Producto' y calcular la inversión total para cada campaña
inversion_por_producto_2018 <- tiendas_2018 %>%
  group_by(Producto) %>%
  summarise(inversion_total = sum(Inversión_Bruta))

# Ordenar las campañas por inversión total en orden descendente
inversion_por_producto_2018 <- inversion_por_producto_2018 %>%
  arrange(desc(inversion_total))

# Mostrar las campañas más importantes
print("Campañas más importantes en 2018:")
print(head(inversion_por_producto_2018, 10))  # Muestra las primeras 10 campañas, puedes ajustar este número según sea necesario


In [ ]:
# Obtener las 10 campañas más importantes del año 2018
campañas_importantes_2018 <- head(inversion_por_producto_2018, 10)

# Filtrar los datos para incluir solo las campañas más importantes
datos_campañas_importantes_2018 <- tiendas_2018 %>%
  filter(Producto %in% campañas_importantes_2018$Producto)

# Agrupar los datos por campaña y medio, y calcular la inversión total para cada combinación
inversion_por_campaña_medio <- datos_campañas_importantes_2018 %>%
  group_by(Producto, Medio, .groups = "drop_last") %>%
  summarise(inversion_total = sum(Inversión_Bruta))

# Calcular el porcentaje de inversión de cada medio en relación con la inversión total de la campaña
inversion_por_campaña_medio <- inversion_por_campaña_medio %>%
  group_by(Producto) %>%
  mutate(porcentaje = (inversion_total / sum(inversion_total)) * 100)

# Ordenar los resultados por inversión total en orden descendente
inversion_por_campaña_medio <- inversion_por_campaña_medio %>%
  arrange(desc(inversion_total))

# Mostrar los resultados
print("Inversión por Medio para las Campañas más importantes en 2018 (en orden descendente):")
print(inversion_por_campaña_medio)


# Comportamiento de la Inversión en TV Abierta

In [ ]:
# Filtrar los datos para incluir solo TV Abierta
tv_abierta <- tiendas_sin_duplicados[tiendas_sin_duplicados$Medio == "TV ABIERTA", ]

# Agrupar los datos por año y calcular la inversión total para cada año
inversion_por_año <- tv_abierta %>%
  group_by(Año) %>%
  summarise(inversion_total = sum(Inversión_Bruta))

# Mostrar la inversión total por año
print("Inversión total en TV Abierta por año:")
print(inversion_por_año)


In [ ]:
# Identificar los tres principales anunciantes en 2018
top_anunciantes_2018 <- tiendas_2018 %>%
  group_by(Anunciante) %>%
  summarise(inversion_total = sum(Inversión_Bruta)) %>%
  arrange(desc(inversion_total)) %>%
  head(3)

# Filtrar los datos para incluir solo las entradas relacionadas con los tres principales anunciantes en 2018
datos_top_anunciantes_2018 <- tiendas_2018[tiendas_2018$Anunciante %in% top_anunciantes_2018$Anunciante, ]

# Agrupar los datos por canal y calcular la inversión total para cada combinación
inversion_top_anunciantes_2018 <- datos_top_anunciantes_2018 %>%
  group_by(Vehículo) %>%
  summarise(inversion_total = sum(Inversión_Bruta))

# Ordenar los resultados por inversión total en orden descendente
inversion_top_anunciantes_2018 <- inversion_top_anunciantes_2018 %>%
  arrange(desc(inversion_total))

# Mostrar los canales más importantes para los tres principales anunciantes en 2018
print("Canales más importantes para los tres principales anunciantes en 2018:")
print(inversion_top_anunciantes_2018)

# Identificar los tres principales anunciantes en 2019
top_anunciantes_2019 <- tiendas_2019 %>%
  group_by(Anunciante) %>%
  summarise(inversion_total = sum(Inversión_Bruta)) %>%
  arrange(desc(inversion_total)) %>%
  head(3)

# Filtrar los datos para incluir solo las entradas relacionadas con los tres principales anunciantes en 2019
datos_top_anunciantes_2019 <- tiendas_2019[tiendas_2019$Anunciante %in% top_anunciantes_2019$Anunciante, ]

# Agrupar los datos por canal y calcular la inversión total para cada combinación
inversion_top_anunciantes_2019 <- datos_top_anunciantes_2019 %>%
  group_by(Vehículo) %>%
  summarise(inversion_total = sum(Inversión_Bruta))

# Ordenar los resultados por inversión total en orden descendente
inversion_top_anunciantes_2019 <- inversion_top_anunciantes_2019 %>%
  arrange(desc(inversion_total))

# Mostrar los canales más importantes para los tres principales anunciantes en 2019
print("Canales más importantes para los tres principales anunciantes en 2019:")
print(inversion_top_anunciantes_2019)
